# Modeling variable Poisson spiking {#sec:variable-spiking}

We have previous modeled spiking as a Poisson process (using an exponential distribution for interspike intervals and a Poisson distribution for the number of spikes in a given interval)

Consider the waiting time for an arrival of a Poisson process that arrives at rate $\beta$. We know that the waiting time is Exponentially distributed.

\begin{align}
f(t\mid \beta) = \beta \mathrm{e}^{-\beta t}.
\end{align}

The probability that a Poisson process has *not* arrived at or before time $t$ is given by the complementary cumulative distribution function (CCDF) of the Exponential distribution.

\begin{align}
P(\text{not arrived by time }t) = \mathrm{e}^{-\beta t}.
\end{align}

Now, consider the case where the arrival rate varies with time; $\beta = \beta(t)$. Consider some small interval, $[0, \Delta t]$. The probability that no Poisson process has arrived in time $\Delta t$ is

\begin{align}
P(\text{not arrived by time } \Delta t) \approx \mathrm{e}^{-\beta(\Delta t) \Delta t},
\end{align}

where $\Delta t$ is small so that $\beta(t)$ does not change appreciably over the time interval and $\beta(t) \approx \beta(t+\Delta t)$. We will soon take the $\Delta t\to 0$ limit, so this approximation is justified.

Now consider another interval of length $\Delta t$ right after the one we just considered. The probability that no process arrives in that interval is

\begin{align}
P(\text{not arrived in time window } [\Delta t, 2\Delta t]) \approx \mathrm{e}^{-\beta(2\Delta t) \Delta t}.
\end{align}

Therefore, owing to the memorylessness of Poisson processes, the probability that no processes arrived in the interval $[0, 2\Delta t]$ is the product of the probabilities of not arriving in the respective subintervals,

\begin{align}
P(\text{not arrived in time } 2\Delta t) \approx \mathrm{e}^{-\beta(\Delta t) \Delta t}\,\mathrm{e}^{-\beta(2\Delta t) \Delta t}.
\end{align}

We can consider $m$ such intervals.

\begin{align}
P(\text{not arrived between in time } m\Delta t) \approx \prod_{k=1}^m \mathrm{e}^{-\beta(k\Delta t) \Delta t} = \exp\left[-\sum_{k=1}^m \Delta t \beta(k\Delta t)\right].
\end{align}

In the limit of $\Delta t\to 0$, the Riemann sum becomes an integral. Taking $t = m\Delta t$, we have

\begin{align}
P(\text{not arrived by time } t) = \exp\left[- \int_0^t\mathrm{d}t'\,\beta(t')\right].
\end{align}

This is the CCDF of the distribution desribing the arrival of a Poisson process with variable rate $\beta(t)$. The CDF is then $1 - \text{CCDF}$,

\begin{align}
f(t\mid \beta(t)) = 1 - \exp\left[- \int_0^t\mathrm{d}t'\,\beta(t')\right].
\end{align}

The probability density function is then

\begin{align}
f(t\mid \beta(t)) &= \frac{\mathrm{d}F}{\mathrm{d}t} = -\frac{\mathrm{d}}{\mathrm{d}t}\,\exp\left[- \int_0^t\mathrm{d}t'\,\beta(t')\right]
= -\exp\left[- \int_0^t\mathrm{d}t'\,\beta(t')\right]\,\frac{\mathrm{d}}{\mathrm{d}t}\left(- \int_0^t\mathrm{d}t'\,\beta(t')\right)\\[1em]
&= \beta(t)\,\exp\left[- \int_0^t\mathrm{d}t'\,\beta(t')\right].
\end{align}

### Multiple arrivals

Say we observe arrivals of a Poisson process one after another. If we have $n$ arrivals of a Poisson process with a variable rate occuring at times $\mathbf{t} = \{t_1, t_2, \ldots, t_n\}$, where the times are ordered, the PDF is given by the product of the PDFs for each waiting time.

\begin{align}
f(\mathbf{t} \mid \beta(t)) = \prod_{i=1}^n\beta(t_i)\exp\left[- \int_{t_{i-1}}^{t_i}\mathrm{d}t'\,\beta(t')\right]
= \left(\prod_{i=1}^n\beta(t_i)\right)\exp\left[- \sum_{i=1}^n\int_{t_{i-1}}^{t_i}\mathrm{d}t'\,\beta(t')\right],
\end{align}

where $t_0$ is when we started observing (it is not a time of an arrival of a Poisson process), and we take $t_0 = 0$. The integrals add together, giving

\begin{align}
f(\mathbf{t} \mid \beta(t)) = \left(\prod_{i=1}^n\beta(t_i)\right)\exp\left[-\int_0^{t_n}\mathrm{d}t'\,\beta(t')\right].
\end{align}

### A finite observation time

In practice, we start observing at time $t = 0$ and end at time $t = T$. While $t_n$ is the time of the last Poisson process we saw arrive, we should also take into account that we continued watching for time $T - t_n$ and saw no arrivals during that time. We therefore have

\begin{align}
f(\mathbf{t}, T;\beta(t)) &= f(\mathbf{t} \mid \beta(t)) \times P(\text{no arrivals between }t_n\text{ and }T) \\[1em]
&= \left(\prod_{i=1}^n\beta(t_i)\right)\exp\left[-\int_0^{t_n}\mathrm{d}t'\,\beta(t')\right]\,\exp\left[\int_{t_n}^T\mathrm{d}t'\,\beta(t')\right] \\[1em]
&= \left(\prod_{i=1}^n\beta(t_i)\right)\exp\left[-\int_0^{T}\mathrm{d}t'\,\beta(t')\right].
\end{align}

### Sampling out this distribution


In [50]:
import numpy as np
import numba

import iqplot

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [53]:
def sample_nonhomogeneous_poisson_process(beta, beta_u, T, beta_args=()):
    samples = []
    tau = 1 / beta_u
    t = 0
    while True:
        t += np.random.exponential(tau)
        r = beta(t, *beta_args) * tau

        if t < T:
            if r > 1:
                raise RuntimeError('beta_u is less than beta; sampling invalid.')
    
            if np.random.uniform() <= r:
                samples.append(t)
        else:
            break

    return np.array(samples)

In [67]:
def sample_nonhomogeneous_poisson_process(beta, beta_j, t_j, beta_args=()):
    """

    Parameters
    ----------
    beta : function, call signature beta(t, *beta_args)
        The function of the rate of arrivals as a function of time.
    beta_j : scalar or array_like
        If scalar, a value of beta that is greater than beta(t)
        for all time. If array_like, then beta_j[j] > beta(t) for
        all times between t_j[j-1] and t_j[j].
    t_j : scalar or array_like
        If scalar, the maximum time of observation. If array_like, must
        be the same length of `beta_j`. beta_j[j] is the value 
        of the the upper bound of the rate for the interval between
        t[j-1] and t[j].

    beta_args : tuple, default ()
        Any other arguments passed to beta.

    Returns
    -------
    output : Numpy array
        Times for arrivals of the nonhomogeneous Poisson process.

    Notes
    -----
    .. This is an implementation of the algorithm on page 86 of 
       Simulation, 5th Ed., by Sheldon Ross, Academic Press, 2013.
    """
    # Convert scalar inputs to arrays
    if np.isscalar(beta_j):
        beta_j = np.array([beta_j])
    if np.isscalar(t_j):
        t_j = np.array([t_j])

    # Make sure dimensions match
    if len(beta_j) != len(t_j):
        raise RuntimeError(
            f'`beta_j` is length {len(beta_j)} '
            f'and `t_j` is length {len(t_j)}. '
            'They must have the same length.'
        )

    return _sample_nonhomogeneous_poisson_process(beta, beta_j, t_j, beta_args)

                                                  
def _sample_nonhomogeneous_poisson_process(beta, beta_j, t_j, beta_args=()):
    """

    Parameters
    ----------
    beta : function, call signature beta(t, *beta_args)
        The function of the rate of arrivals as a function of time.
    beta_j : Numpy array
        If scalar, a value of beta that is greater than beta(t)
        for all time. If array_like, then beta_j[j] > beta(t) for
        all times between t_j[j-1] and t_j[j].
    t_j : Numpy array
        Must be the same length of `beta_j`. beta_j[j] is the value 
        of the the upper bound of the rate for the interval between
        t[j-1] and t[j].

    beta_args : tuple, default ()
        Any other arguments passed to beta.

    Returns
    -------
    output : Numpy array
        Times for arrivals of the nonhomogeneous Poisson process.

    Notes
    -----
    .. This is an implementation of the algorithm on page 86 of 
       Simulation, 5th Ed., by Sheldon Ross, Academic Press, 2013.
    """
    # Number of samples to take before concatenating arrays
    n_samples = 1000

    # Initializations
    t = 0.0  # time
    j = 0    # Index in beta_j and t_j arrays
    i = 0    # index in sample array
    n = 0    # total number of samples drawn
    x_from_boundary = False  # If we've hit a boundary of

    # Array for storing subtrajectory
    samples = np.empty(n_samples, dtype=float)

    # Output array for all samples
    samples_output = np.array([], dtype=float)

    # Loop until done (we exceed final time point)
    not_done = True
    while not_done:
        # Take samples until we fill array
        # We do it this way for speed to avoid list append operations
        while not_done and i < n_samples:
            if x_from_boundary:
                x = (x - t_j[j] + t) * beta_j[j] / beta_j[j+1]
                t = t_j[j]
                j += 1
            else:
                x = np.random.exponential(1 / beta_j[j])

            if t + x > t_j[j]:
                # If we got here, we went past the edge of this interval
                if j == len(t_j) - 1:
                    not_done = False
                else:
                    x_from_boundary = True
            else:
                t += x
                x_from_bounday = False
                if np.random.uniform() <= beta(t, *beta_args) / beta_j[j]:
                    samples[i] = t
                    i += 1
                    n += 1
        samples_output = np.concatenate((samples_output, samples))
        i = 0

    return np.array(samples_output[:n])

In [72]:
beta = lambda t: 1.15 + np.sin(t/10)
beta_j = 2.15
t_j = 500
samples = sample_nonhomogeneous_poisson_process(
    beta, 
    beta_j, 
    t_j
)

In [69]:
p = iqplot.strip(samples, marker='dash', marker_kwargs=dict(alpha=0.3))
bokeh.io.show(p)

In [75]:
len(samples)

543

In [73]:
%timeit sample_nonhomogeneous_poisson_process(beta, beta_j, t_j)

1.84 ms ± 18.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [74]:
%timeit sample_variable_poisson(lambda t: 1.15 + np.sin(t/10), 2.15, T)

1.25 ms ± 13 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:
T = 500
samples = sample_variable_poisson(lambda t: 1.15 + np.sin(t/10), 2.5, T)

p = iqplot.strip(samples, marker='dash', marker_kwargs=dict(alpha=0.3))
bokeh.io.show(p)

In [55]:
len(samples)

549